### Llama-3.2-3B-Instruct (fine-tuning using unsloth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pip install -r requirements.txt

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


True
1
Tesla T4


### Loading the model and Tokenizer for fine-tuning

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage

model_id = "unsloth/Llama-3.2-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id ,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

### We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None,
)

Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

### Data Transformation done

In [ ]:
import pandas as pd
from datasets import Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

# Step 2: Load CSV and prepare data in the expected format
df = pd.read_csv("final_data.csv")
data = []


system_prompt = """
You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.

For any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.

However, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:

'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'

Your response should always focus on Python and topics covered by Upflairs.
"""
for i in range(df.shape[0]):
    sample = {
        'conversations': [
            {"from": "system", "value": system_prompt},
            {'from': 'human', 'value': df.loc[i, "Question"]},
            {'from': 'gpt', 'value': df.loc[i, "Answer"]}
        ]
    }
    data.append(sample)

print("No. of samples in the dataset:", len(data))

# Step 3: Convert list of dictionaries to a Dataset object
dataset = Dataset.from_list(data)

# Step 4: Apply the standardization function
dataset = standardize_sharegpt(dataset)

# Step 5: Define the formatting function to apply the chat template
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

# Apply the formatting function with map
dataset = dataset.map(formatting_prompts_func, batched=True)

# Optional: View a sample of the formatted dataset
print(dataset[0])


No. of samples in the dataset: 39


Standardizing format:   0%|          | 0/39 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

{'conversations': [{'content': "\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n", 'role': 'system'}, {'content': 'tell me about upflairs?', 'role': 'user'}, {'content': "UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging techn

In [ ]:
print(dataset[2])

{'conversations': [{'content': "\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n", 'role': 'system'}, {'content': 'How can I enroll in a course?', 'role': 'user'}, {'content': "You can enroll in any course by visiting the Upflairs website, selecting the course you're interested in, and clicking the 'Enro

<a name="Train"></a>
### configuring training arguments with  `SFTTrainer`

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# set the hyperparameters for supervised fine-tuning
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 15,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/39 [00:00<?, ? examples/s]

Unsloth Trainer initialaizing, so that we can fine-tune SFTTrainer with unsloth.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

In [ ]:
print("No. of Documents after tokenized : ",len(trainer.train_dataset))

No. of Documents after tokenized :  39


In [ ]:
# each query has 3 keys
trainer.train_dataset[5].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

We verify masking on data:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:\n\n'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.'\n\nYour response should always focus on Python and topics covered by Upflairs.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow long is the internship program at Upflairs?<|eot_id|><|start_header_

We can see the System and Instruction prompts are successfully masked!

In [ ]:
trainer_stats = trainer.train()   # start the fine-tuning with epochs=1

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 39 | Num Epochs = 12
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.421700
2,1.530200
3,1.121700
4,1.835300
5,1.491900
6,1.057800
7,0.984900
8,0.959600
9,0.970100
10,0.568000


### Ask the query to the fine-tuned pre-loaded model

In [ ]:
## ask the query after the training model
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",)

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

messages = [
    {"role":"system","content":"You are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.Your response should always focus on Python and topics covered by Upflairs."},
    {"role": "user", "content": "tell me about upflairs?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1, min_p = 0.1)
tokenizer.batch_decode(outputs)   # output with masking

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a specialized assistant for Upflairs, dedicated to answering queries related strictly to Python programming. Upflairs offers courses in Data Science, Machine Learning, DevOps, Full Stack Development, IoT, and System Embedding, all focused on Python.\n\nFor any Python-related question, respond with clear, accurate explanations, code snippets, or examples as needed.\n\nHowever, if a question involves any programming language other than Python (like Java, C++, or others), reply with this message:'I am here to assist with Python-related programming questions only. For inquiries about other programming languages, please consult other resources.Your response should always focus on Python and topics covered by Upflairs.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me about upflairs?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nUp

In [ ]:
# Feteched Data from masked output
response = tokenizer.batch_decode(outputs)
print("Robo response : ",response[0].split('\n')[-1]) # extracted exact response from the chat response


Robo response :  UpFlairs is an innovative educational technology company dedicated to empowering students across India. With a focus on emerging technologies like AI/ML, Data Science, Cloud computing, DevOps, Full Stack Web Development, Embedded Systems, IoT and Robotics. We've educated over 50K+ students worldwide, including those from prestigious institutions


### Asking multiple queries to fine-tuned pre-loaded model

In [ ]:
# Define a list of testing queries (we can also add system prompt for that follow prevous one  inferencing)
test_queries = [
    {"role": "user", "content": "tell me about upflairs."},
    {"role": "user", "content": "What courses does Upflairs offer?"},
    {"role": "user", "content": "How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?"},
    {"role": "user", "content": "write a program to print hello world."},
    {"role": "user", "content": "write a program to print hello world in python."},
    {"role": "user", "content": "write a program to print hello world in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in c++?"},
    {"role": "user", "content": "how we can find the average of the integer array in java?"},
    {"role": "user", "content": "how we can find the average of the integer array in python?"}
]

# Process each query using the chat template and tokenize
for query in test_queries:
    inputs = tokenizer.apply_chat_template(
        [query],
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate responses with varied parameters for testing
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=64,
        use_cache=True,
        temperature=0.7,  # Lowered temperature for more deterministic responses
        min_p=0.1
    )

    # Decode and print each output
    response = tokenizer.batch_decode(outputs)
    print(f"Query: {query['content']}")
    print("Robo response : ",response[0].split('\n')[-1][:-10]) # extracted exact response from the chat response
    print()


Query: tell me about upflairs.
Robo response :  UpFlairs is an online education platform that provides courses and degree programs in fields such as business, technology, and creative arts. They offer flexible and affordable options for students, allowing them to earn a degree from the comfort of their own homes. UpFlairs also provides career counseling and support to help students advanc

Query: What courses does Upflairs offer?
Robo response :  UpFlairs offers courses in AI/ML, Data Science, Cloud computing, Cyber Security, Web Development, Digital Marketing, and more.

Query: How do you use list comprehension at Upflairs to create a list of even numbers from 1 to 100?
Robo response :  seen_numbers = [num for num in range(1, 101) if num % 2 == 0]

Query: write a program to print hello world.
Robo response :  print('Hello World')

Query: write a program to print hello world in python.
Robo response :  print('Hello World')

Query: write a program to print hello world in java?
Robo resp

### saving the model on a local system

In [ ]:
## saving the model into google drive
import os
os.chdir("/content/drive/MyDrive/llama fine-tuning")
os.makedirs("Lora_saved_model",exist_ok=True)
model_dir = os.path.join("/content/drive/MyDrive/llama fine-tuning","new_saved_model")
model_dir

'/content/drive/MyDrive/llama fine-tuning/new_saved_model'

In [ ]:
## saving the model
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)
print("Saved model at your located address : ",model_dir)

Saved model at your located address :  /content/drive/MyDrive/llama fine-tuning/new_saved_model


### online saving into Huggingface Hub
<ul>
<li>create model  repository first on huggingface.</li>
<li>create access token for that particular repository.</li>
<li>make sure assign the write permission to the model.</li>
</ul>

In [ ]:
from huggingface_hub import login

# Log in to Hugging Face
huggingface_token = ""
login()

print("Successfully logged into Hugging Face!")

# Push model and tokenizer to the hub
model.push_to_hub("Ranjit123321/llama-unslothfinetuned", token=huggingface_token)
tokenizer.push_to_hub("Ranjit123321/llama-unslothfinetuned", token=huggingface_token)


### loading from the local, to inference, change the parameter False to True

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "How do you write a Python function at Upflairs to check if a string is a palindrome?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

####  loading model from hugging face for inferencing

In [ ]:
from huggingface_hub import login

# Log in to Hugging Face
huggingface_token = "###################"  # for  Ranjit123321/llama-unslothfinetuned
login(huggingface_token)
print("Successfully logged into Hugging Face!")

In [2]:
## model loaded, and raised the query both code

from transformers import AutoTokenizer, BitsAndBytesConfig, TextStreamer
from peft import AutoPeftModelForCausalLM

# Model ID
model_id = "Ranjit123321/llama-unslothfinetuned"

# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  #
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load the model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


model.eval()

# Function to generate a response from the model
def query_model(prompt):
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")


    streamer = TextStreamer(tokenizer)

    # Generate the response
    output = model.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=128,
        temperature=1.0,
        top_p=0.9,           #
        streamer=streamer
    )

    # Decode and return the response
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example query
prompt = "How do you write a Python function to check if a string is a palindrome?"
response = query_model(prompt)
print("Response from model:")
print(response)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

<|begin_of_text|>How do you write a Python function to check if a string is a palindrome??

def is_palindrome(s):
	if len(s) <= 1:
		return True
	else:
		if s[0] == s[-1]:
			return is_palindrome(s[1:-1])
		else:
			return False
print(is_palindrome('radar'))  # True
print(is_palindrome('python'))  # False
print(is_palindrome(''))  # True
print(is_palindrome('a'))  # True
print(is_palindrome('ab'))  # False
print(is_palindrome('madam'))  # True
print(is_palindrome('hello'))  # False

Response from model:
How do you write a Python function to check if a string is a palindrome??

def is_palindrome(s):
	if len(s) <= 1:
		return True
	else:
		if s[0] == s[-1]:
			return is_palindrome(s[1:-1])
		else:
			return False
print(is_palindrome('radar'))  # True
print(is_palindrome('python'))  # False
print(is_palindrome(''))  # True
print(is_palindrome('a'))  # True
print(is_palindrome('ab'))  # False
print(is_palindrome('madam'))  # True
print(is_palindrome('hello'))  # False



In [9]:
# Example query
prompt = "write program to determine an integer number whether it is an even or odd."
response = query_model(prompt)

print()
print("Response from model  : ")
print(response)

<|begin_of_text|>write program to determine an integer number whether it is an even or odd. class OddOrEven {
	public static void main(String[] args) {
		int number = 10;
		if (number % 2 == 0) {
			System.out.println(number + " is an even number.");
		} else {
			System.out.println(number + " is an odd number.");
		}
	}
}
If this code is correct because it uses the modulo operator to determine if the number is even or odd and if so, it prints the number and the corresponding type of number. In the next example, we will show you another code for checking whether a given integer is an even or odd number. class

Response from model  : 
write program to determine an integer number whether it is an even or odd. class OddOrEven {
	public static void main(String[] args) {
		int number = 10;
		if (number % 2 == 0) {
			System.out.println(number + " is an even number.");
		} else {
			System.out.println(number + " is an odd number.");
		}
	}
}
If this code is correct because it uses the modulo

#### inferencing with system prompt

In [19]:
# Function to generate a response from the model
def query_model(system_message, user_message):
    # Combine system and user messages in the prompt
    prompt = f"System: {system_message}\nUser: {user_message}\nModel:"

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate the response
    output = model.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=128,
        temperature=1.0,
        top_p=0.9  # Optional for sampling randomness
    )

    # Decode and return the response
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example query
system_prompt = """You are a coding assistant specializing **only in Python programming**, particularly within the EdTech domain. Your responses should address both theoretical and coding aspects only when relevant, ensuring examples remain simple and beginner-friendly. If the question is unrelated to Python, politely respond with:
    'I only provide support for Python programming topics. Please ask something related to Python.'"""

user_message = "write a program to determine an integer whether it is an even or odd in java?"
response = query_model(system_prompt, user_message)

print("Response from the model:")
print(response)


Response from the model:
System: You are a coding assistant specializing **only in Python programming**, particularly within the EdTech domain. Your responses should address both theoretical and coding aspects only when relevant, ensuring examples remain simple and beginner-friendly. If the question is unrelated to Python, politely respond with:
    'I only provide support for Python programming topics. Please ask something related to Python.'
User: write a program to determine an integer whether it is an even or odd in java?
Model: You'll respond with a simple 'True' or 'False'.

True
I only provide support for Python programming topics. Please ask something related to Python.


In [20]:
# Example query
system_prompt = """You are a coding assistant specializing **only in Python programming**, particularly within the EdTech domain. Your responses should address both theoretical and coding aspects only when relevant, ensuring examples remain simple and beginner-friendly. If the question is unrelated to Python, politely respond with:
    'I only provide support for Python programming topics. Please ask something related to Python.'"""

user_message = "write a program to determine an integer whether it is an even or odd in java?"
response = query_model(system_prompt, user_message)

print("Response from the model:")
print(response)

Response from the model:
System: You are a coding assistant specializing **only in Python programming**, particularly within the EdTech domain. Your responses should address both theoretical and coding aspects only when relevant, ensuring examples remain simple and beginner-friendly. If the question is unrelated to Python, politely respond with:
    'I only provide support for Python programming topics. Please ask something related to Python.'
User: write a program to determine an integer whether it is an even or odd in java?
Model: Since you're focusing on Python, I can help with that specific Java code if you'd like. Here's a simple solution using Java: def is_even(n): return n % 2 == 0
print(is_even(10))
I only provide support for Python programming topics. Please ask something related to Python.

Welcome to Django Girls! You're now part of an exclusive community of awesome women who love Python and web development.
Welcome to Django Girls! You're now part of an exclusive community 

### Thank You